In [2]:
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet34, ResNet34_Weights
from torchvision import transforms
from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score
import logging
from datetime import datetime
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pandas as pd
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

In [3]:

torch.cuda.empty_cache()

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
test_logger = logging.getLogger("test_logger")
test_logger.setLevel(logging.INFO)
file_handler = logging.FileHandler("test_results.log", mode="w")
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
test_logger.addHandler(file_handler)

In [6]:

num_epochs = 20
batch_size = 8
learning_rate = 1e-4



In [7]:
# def get_train_augmentations():
#     return A.Compose([
#         A.Resize(256, 256),
#         # Geometric transformations
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.Rotate(
#                 limit=359,
#                 border_mode=cv2.BORDER_CONSTANT,
#                 value=0,          
#                 mask_value=0,     
#                 p=0.5
#             ),
#         # Noise and blur
#         A.GaussNoise(
#                 var_limit=(0.0005, 0.003),  
#                 mean=0.0,                   
#                 p=0.5
#             ),
#         A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.2),
#         A.GaussianBlur(blur_limit=(3, 7), p=0.3),
#         A.MotionBlur(blur_limit=7, p=0.2),
#         # Occlusion and artifacts
#         A.CoarseDropout(max_holes=8, max_height=32, max_width=32, 
#                        min_holes=1, min_height=8, min_width=8, 
#                        fill_value=0, p=0.3),
#         # Normalization
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2(),
#     ], additional_targets={'mask': 'mask'})


In [16]:
# class PolypDataset(Dataset):
#     def __init__(self, image_dir, mask_dir, transform=None):
#         self.image_paths = sorted(glob.glob(os.path.join(image_dir, "*.*")))
#         self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.*")))
#         assert len(self.image_paths) == len(self.mask_paths), \
#             f"Number of images ({len(self.image_paths)}) and masks ({len(self.mask_paths)}) don't match!"
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = cv2.cvtColor(cv2.imread(self.image_paths[idx]), cv2.COLOR_BGR2RGB)
#         mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        
#         if self.transform:
#             transformed = self.transform(image=image, mask=mask)
#             image = transformed["image"]
#             mask = transformed["mask"]
#             # Ensure mask is float32 and in [0,1] range
#             mask = mask.float() / 255.0
#         else:
#             image = cv2.resize(image, (256, 256)) / 255.0
#             mask = (cv2.resize(mask, (256, 256)) > 127).astype(np.float32)  # Changed to float32
#             image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
#             image = torch.from_numpy(image.transpose(2, 0, 1)).float()
#             mask = torch.from_numpy(mask).unsqueeze(0).float()  # Ensure float
            
#         filename = os.path.basename(self.image_paths[idx])
#         return image, mask, filename

def metrics(y_true, y_pred):
    # Convert to numpy arrays if they're tensors
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.cpu().numpy()
    
    # Flatten and threshold predictions to binary (0 or 1)
    y_true = y_true.flatten().astype(int)  # Ensure it's integer type
    y_pred = (y_pred.flatten() > 0.5).astype(int)  # Threshold at 0.5
    
    # Calculate metrics
    iou = jaccard_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    prec = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return iou, rec, prec, f1

class DiceBCELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCELoss()

    def forward(self, pred, target):
        # Ensure target is float (in case it's not)
        target = target.float()
        
        smooth = 1.0
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        
        # BCE Loss
        bce_loss = self.bce(pred, target)
        
        # Dice Loss
        intersection = (pred * target).sum()
        dice_coeff = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
        dice_loss = 1 - dice_coeff
        
        return dice_loss + bce_loss

# def train_one_epoch(model, dataloader, criterion, optimizer, device, writer, epoch, dataset_name):
#     model.train()
#     running_loss = 0.0

#     all_y_true = []
#     all_y_pred = []

#     for images, masks, _ in tqdm(dataloader, desc="Training", leave=False):
#         images = images.to(device)
#         masks = masks.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, masks)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)

#         preds = (outputs > 0.5).float().detach()
#         all_y_true.append(masks.cpu().numpy())
#         all_y_pred.append(preds.cpu().numpy())

#     epoch_loss = running_loss / len(dataloader.dataset)
#     all_y_true = np.concatenate(all_y_true, axis=0)
#     all_y_pred = np.concatenate(all_y_pred, axis=0)

#     iou, recall, precision, f1 = metrics(all_y_true, all_y_pred)

#     writer.add_scalar(f"Loss/Train/{dataset_name}", epoch_loss, epoch)
#     writer.add_scalar(f"Metrics/Train_IoU/{dataset_name}", iou, epoch)
#     writer.add_scalar(f"Metrics/Train_Recall/{dataset_name}", recall, epoch)
#     writer.add_scalar(f"Metrics/Train_Precision/{dataset_name}", precision, epoch)
#     writer.add_scalar(f"Metrics/Train_F1/{dataset_name}", f1, epoch)
#     torch.cuda.empty_cache()
#     return epoch_loss, iou, recall, precision, f1

def validate(model, dataloader, criterion, device, writer, epoch, dataset_name):
    model.eval()
    running_loss = 0.0
    all_y_true = []
    all_y_pred = []

    with torch.no_grad():
        for images, masks, _ in tqdm(dataloader, desc="Validation", leave=False):
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)

            running_loss += loss.item() * images.size(0)
            preds = (outputs > 0.5).float().detach()
            all_y_true.append(masks.cpu().numpy())
            all_y_pred.append(preds.cpu().numpy())

    val_loss = running_loss / len(dataloader.dataset)
    all_y_true = np.concatenate(all_y_true, axis=0)
    all_y_pred = np.concatenate(all_y_pred, axis=0)
    iou, recall, precision, f1 = metrics(all_y_true, all_y_pred)

    writer.add_scalar(f"Loss/Val/{dataset_name}", val_loss, epoch)
    writer.add_scalar(f"Metrics/Val_IoU/{dataset_name}", iou, epoch)
    writer.add_scalar(f"Metrics/Val_Recall/{dataset_name}", recall, epoch)
    writer.add_scalar(f"Metrics/Val_Precision/{dataset_name}", precision, epoch)
    writer.add_scalar(f"Metrics/Val_F1/{dataset_name}", f1, epoch)

    evaluate_and_save(model, dataloader, criterion, device, save_csv_path=f"results/val_predictions.csv", threshold=0.5)

    return val_loss, iou, recall, precision, f1

def test_model(model, dataloader, criterion, device, save_folder="predicted_masks"):
    model.eval()
    running_loss = 0.0
    all_y_true = []
    all_y_pred = []

    os.makedirs(save_folder, exist_ok=True)

    with torch.no_grad():
        for images, masks, fnames in tqdm(dataloader, desc="Testing", leave=False):
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)

            loss = criterion(outputs, masks)
            running_loss += loss.item() * images.size(0)

            preds = (outputs > 0.5).float().detach()
            all_y_true.append(masks.cpu().numpy())
            all_y_pred.append(preds.cpu().numpy())

            preds_np = preds.cpu().numpy()
            for i in range(preds_np.shape[0]):
                pred_mask = (preds_np[i, 0] * 255).astype(np.uint8)
                save_path = os.path.join(save_folder, fnames[i])
                cv2.imwrite(save_path, pred_mask)

    test_loss = running_loss / len(dataloader.dataset)
    all_y_true = np.concatenate(all_y_true, axis=0)
    all_y_pred = np.concatenate(all_y_pred, axis=0)

    iou, recall, precision, f1 = metrics(all_y_true, all_y_pred)

    evaluate_and_save(model, dataloader, criterion, device, save_csv_path=f"results/test_predictions.csv", threshold=0.5)

    return test_loss, iou, recall, precision, f1


In [17]:
# dataset = {"name": "40_60", 
#      "train_image_dir": 'C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/train/40_60',
#      "train_mask_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Masks/train/40_60",
#      "val_image_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/val",
#      "val_mask_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Masks/val",
#      "test_image_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/test",
#      "test_mask_dir": "C:/Users/Dara/Desktop/Final_thesis//data_choosing/Masks/test"}


     


In [18]:


# results = []
# encoder_name = "resnet34"  


# dataset_name = dataset["name"]
# print(f"Training on dataset: {dataset_name}")

# train_dataset = PolypDataset(dataset["train_image_dir"], dataset["train_mask_dir"], transform=get_train_augmentations())
# val_dataset = PolypDataset(dataset["val_image_dir"], dataset["val_mask_dir"])
# test_dataset = PolypDataset(dataset["test_image_dir"], dataset["test_mask_dir"])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# model = smp.UnetPlusPlus(
#     encoder_name=encoder_name,
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1,
#     activation="sigmoid"
# )
# model = model.to(device)

# criterion = DiceBCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# log_dir = f"logs/{dataset_name}_{encoder_name}"
# writer = SummaryWriter(log_dir=log_dir)

# patience = 10
# best_val_iou = 0.0
# best_epoch = -1
# epochs_without_improvement = 0

# for epoch in range(num_epochs):
#     print(f"Эпоха {epoch+1}/{num_epochs} для датасета {dataset_name}")
    
#     train_loss, train_iou, train_recall, train_precision, train_f1 = train_one_epoch(
#         model, train_loader, criterion, optimizer, device, writer, epoch, dataset_name)
#     print(f"Train Loss: {train_loss:.4f} | IoU: {train_iou:.4f} | Recall: {train_recall:.4f} | Precision: {train_precision:.4f} | F1: {train_f1:.4f}")
    
#     val_loss, val_iou, val_recall, val_precision, val_f1 = validate(
#         model, val_loader, criterion, device, writer, epoch, dataset_name)
#     print(f"Val Loss: {val_loss:.4f} | IoU: {val_iou:.4f} | Recall: {val_recall:.4f} | Precision: {val_precision:.4f} | F1: {val_f1:.4f}")

#     if val_iou > best_val_iou:
#         best_val_iou = val_iou
#         best_epoch = epoch
#         torch.save(model.state_dict(), f"unetplusplus_{dataset_name}_best_model.pth")
#         print("Новая лучшая модель сохранена.")
#         epochs_without_improvement = 0
#     else:
#         epochs_without_improvement += 1
#         if epochs_without_improvement >= patience:
#             print(f"Ранняя остановка на эпохе {epoch+1}")
#             break

# print(f"Обучение завершено для датасета {dataset_name}. Лучшая модель на эпохе {best_epoch+1} с IoU: {best_val_iou:.4f}")

# model.load_state_dict(torch.load(f"unetplusplus_{dataset_name}_best_model.pth"))
# test_loss, test_iou, test_recall, test_precision, test_f1 = test_model(
#     model, test_loader, criterion, device, save_folder=f"predicted_masks_{dataset_name}")

# results.append({
#     "Dataset": dataset_name,
#     "Best Epoch": best_epoch + 1,
#     "Test IoU": test_iou,
#     "Test Recall": test_recall,
#     "Test Precision": test_precision,
#     "Test F1": test_f1,
#     "Test Loss": test_loss
# })

# writer.close()

# results_df = pd.DataFrame(results)
# print(results_df)

In [19]:
# def test_model_with_ground_truth(model, dataloader, criterion, device, save_folder=".", threshold=0.5):
#     model.eval()
#     running_loss = 0.0
#     all_pred_data = []
    
#     if not os.path.exists(save_folder):
#         os.makedirs(save_folder)

#     with torch.no_grad():
#         for images, masks, filenames in tqdm(dataloader, desc="Testing"):
#             images = images.to(device)
#             masks = masks.to(device)

#             outputs = model(images)
#             loss = criterion(outputs, masks)
#             running_loss += loss.item() * images.size(0)

#             preds = outputs.float()
#             preds_np = preds.cpu().numpy()
#             masks_np = masks.cpu().numpy()  # Ground truth masks
            
#             for i in range(preds_np.shape[0]):
#                 mask_pred = preds_np[i, 0]
#                 mask_true = masks_np[i, 0]  # Ground truth mask
                
#                 # Save predicted masks
#                 mask_img = ((mask_pred > threshold) * 255).astype(np.uint8)
#                 cv2.imwrite(os.path.join(save_folder, filenames[i]), mask_img)
                
#                 # Collect all data points (not sampled)
#                 h, w = mask_pred.shape
#                 for y in range(h):
#                     for x in range(w):
#                         # Only save points where true value is 1 (or all if you prefer)
#                         # To save all points, remove the if condition
#                         if mask_true[y, x] == 1:  # This saves only True values
#                             all_pred_data.append({
#                                 'filename': filenames[i],
#                                 'x': x,
#                                 'y': y,
#                                 'pred_value': float(mask_pred[y, x]),
#                                 'true_value': int(mask_true[y, x]),  # 0 or 1
#                                 'above_threshold': int(mask_pred[y, x] > threshold)
#                             })

#     # Save all data
#     preds_df = pd.DataFrame(all_pred_data)
#     preds_csv_path = os.path.join(save_folder, 'predictions_with_ground_truth.csv')
#     preds_df.to_csv(preds_csv_path, index=False)
#     print(f"Data with predictions and ground truth saved to {preds_csv_path}")

#     # Calculate metrics
#     test_loss = running_loss / len(dataloader.dataset)
    
#     # Calculate metrics for different thresholds
#     thresholds = 0.5
#     metrics_df = pd.DataFrame(columns=['threshold', 'precision', 'recall', 'f1'])
    
#     for thresh in thresholds:
#         preds_df['binary_pred'] = (preds_df['pred_value'] > thresh).astype(int)
#         precision = precision_score(preds_df['true_value'], preds_df['binary_pred'], zero_division=0)
#         recall = recall_score(preds_df['true_value'], preds_df['binary_pred'], zero_division=0)
#         f1 = f1_score(preds_df['true_value'], preds_df['binary_pred'], zero_division=0)
        
#         metrics_df = pd.concat([metrics_df, pd.DataFrame({
#             'threshold': [thresh],
#             'precision': [precision],
#             'recall': [recall],
#             'f1': [f1]
#         })], ignore_index=True)
    
#     metrics_csv_path = os.path.join(save_folder, 'metrics_by_threshold.csv')
#     metrics_df.to_csv(metrics_csv_path, index=False)
#     print(f"Metrics for different thresholds saved to {metrics_csv_path}")

#     return test_loss, metrics_df

In [20]:
# results = []
# encoder_name = "resnet34"  

# dataset_name = dataset["name"]
# print(f"Training on dataset: {dataset_name}")

# train_dataset = PolypDataset(dataset["train_image_dir"], dataset["train_mask_dir"], transform=get_train_augmentations())
# val_dataset = PolypDataset(dataset["val_image_dir"], dataset["val_mask_dir"])
# test_dataset = PolypDataset(dataset["test_image_dir"], dataset["test_mask_dir"])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# model = smp.UnetPlusPlus(
#     encoder_name=encoder_name,
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1,
#     activation="sigmoid"
# )
# model = model.to(device)

# criterion = DiceBCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# log_dir = f"logs/{dataset_name}_{encoder_name}"
# writer = SummaryWriter(log_dir=log_dir)

# patience = 10
# best_val_iou = 0.0
# best_epoch = -1
# epochs_without_improvement = 0

# # Load the best model
# model.load_state_dict(torch.load(f"unetplusplus_{dataset_name}_best_model.pth"))



In [21]:
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet34, ResNet34_Weights
from torchvision import transforms
from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score
import logging
from datetime import datetime
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pandas as pd
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("visualizations", exist_ok=True)

def visualize_samples_with_names(original_images, original_masks, augmented_images, augmented_masks, filenames, epoch=0, prefix="train"):
    import matplotlib.pyplot as plt
    import numpy as np

    def denormalize(tensor):
        mean = torch.tensor([0.485, 0.456, 0.406], device=tensor.device).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=tensor.device).view(3, 1, 1)
        return (tensor * std + mean).clamp(0, 1)

    def to_uint8(tensor_img):
        img = tensor_img.permute(1, 2, 0).cpu().numpy()
        img = (img * 255).astype(np.uint8)
        return img

    num_samples = min(5, len(original_images))
    fig, axes = plt.subplots(num_samples, 4, figsize=(18, num_samples * 4))

    if num_samples == 1:
        axes = axes.reshape(1, -1)

    for i in range(num_samples):
        # Original Image
        orig_img = (original_images[i].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
        axes[i, 0].imshow(orig_img)
        axes[i, 0].set_title(f"Original Image\n{filenames[i]}")
        axes[i, 0].axis('off')

        orig_mask = original_masks[i].cpu().numpy()
        if orig_mask.ndim == 3:
            orig_mask = orig_mask[0]
        axes[i, 1].imshow(orig_mask, cmap='gray')
        axes[i, 1].set_title("Original Mask")
        axes[i, 1].axis('off')

        aug_img = to_uint8(denormalize(augmented_images[i]))
        axes[i, 2].imshow(aug_img)
        axes[i, 2].set_title(f"Augmented Image\n{filenames[i]}")
        axes[i, 2].axis('off')

        aug_mask = augmented_masks[i].cpu().numpy()
        if aug_mask.ndim == 3:
            aug_mask = aug_mask[0]
        axes[i, 3].imshow(aug_mask, cmap='gray')
        axes[i, 3].set_title("Augmented Mask")
        axes[i, 3].axis('off')

    plt.tight_layout()
    save_path = f"visualizations/{prefix}_epoch_{epoch}_samples.png"
    plt.savefig(save_path)
    print(f" Visualization saved to: {save_path}")
    plt.show()
    plt.close()

    def denormalize(tensor):
        mean = torch.tensor([0.485, 0.456, 0.406], device=tensor.device).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=tensor.device).view(3, 1, 1)
        return (tensor * std + mean).clamp(0, 1)

    num_samples = min(5, len(original_images))
    fig, axes = plt.subplots(num_samples, 4, figsize=(18, num_samples * 4))

    if num_samples == 1:
        axes = axes.reshape(1, -1)

    for i in range(num_samples):
        # --- Original Image ---
        axes[i, 0].imshow(original_images[i].permute(1, 2, 0).cpu().numpy())
        axes[i, 0].set_title(f"Original Image\n{filenames[i]}")
        axes[i, 0].axis('off')

        # --- Original Mask ---
        orig_mask = original_masks[i].cpu().numpy()
        if orig_mask.ndim == 3:
            orig_mask = orig_mask[0]
        axes[i, 1].imshow(orig_mask, cmap='gray')
        axes[i, 1].set_title("Original Mask")
        axes[i, 1].axis('off')

        # --- Augmented Image ---
        aug_img = denormalize(augmented_images[i]).permute(1, 2, 0).cpu().numpy()
        aug_img = (aug_img * 255).astype(np.uint8)  # <--- ЭТО ДОБАВИТЬ
        axes[i, 2].imshow(aug_img)
        axes[i, 2].set_title(f"Augmented Image\n{filenames[i]}")
        axes[i, 2].axis('off')

        # --- Augmented Mask ---
        aug_mask = augmented_masks[i].cpu().numpy()
        if aug_mask.ndim == 3:
            aug_mask = aug_mask[0]
        axes[i, 3].imshow(aug_mask, cmap='gray')
        axes[i, 3].set_title("Augmented Mask")
        axes[i, 3].axis('off')

    plt.tight_layout()
    plt.savefig(f"visualizations/{prefix}_epoch_{epoch}_samples.png")
    plt.close()


def get_train_augmentations():
    return A.Compose([
        A.Resize(256, 256),

        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=180, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=0.6),
        A.GaussNoise(var_limit=(1.0, 3.0), mean=0.0, p=0.1),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), per_channel=True, p=0.2),
        A.GaussianBlur(blur_limit=(3, 5), p=0.3),
        A.CoarseDropout(
            max_holes=2,
            max_height=16,
            max_width=16,
            min_holes=1,
            min_height=8,
            min_width=8,
            fill_value=0,
            p=0.1
        ),

        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ], additional_targets={'mask': 'mask'})


def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406], device=tensor.device).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225], device=tensor.device).view(3, 1, 1)
    return (tensor * std + mean).clamp(0, 1)


class PolypDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, "*.*")))
        self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.*")))
        assert len(self.image_paths) == len(self.mask_paths), \
            f"Number of images ({len(self.image_paths)}) and masks ({len(self.mask_paths)}) don't match!"
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.image_paths[idx]), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
            # Ensure mask is float32 and in [0,1] range
            mask = mask.float() / 255.0
        else:
            image = cv2.resize(image, (256, 256)) / 255.0
            mask = (cv2.resize(mask, (256, 256)) > 127).astype(np.float32)  # Changed to float32
            image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            image = torch.from_numpy(image.transpose(2, 0, 1)).float()
            mask = torch.from_numpy(mask).unsqueeze(0).float()  # Ensure float
            
        filename = os.path.basename(self.image_paths[idx])
        return image, mask, filename


def analyze_prediction(pred, mask, threshold=0.5):
    pred_bin = (pred > threshold).astype(np.uint8)
    mask_bin = mask.astype(np.uint8)

    total = pred.size
    tp = ((pred_bin == 1) & (mask_bin == 1)).sum()
    tn = ((pred_bin == 0) & (mask_bin == 0)).sum()
    fp = ((pred_bin == 1) & (mask_bin == 0))
    fn = ((pred_bin == 0) & (mask_bin == 1))

    fp_vals = pred[fp]
    fn_vals = pred[fn]

    return {
        "tp_percent": round(tp / total * 100, 2),
        "tn_percent": round(tn / total * 100, 2),
        "fp_percent": round(fp.sum() / total * 100, 2),
        "fn_percent": round(fn.sum() / total * 100, 2),
        "fp_pred_mean": round(fp_vals.mean() if fp_vals.size > 0 else 0.0, 4),
        "fp_pred_min": round(fp_vals.min() if fp_vals.size > 0 else 0.0, 4),
        "fp_pred_max": round(fp_vals.max() if fp_vals.size > 0 else 0.0, 4),
        "fn_pred_mean": round(fn_vals.mean() if fn_vals.size > 0 else 0.0, 4),
        "fn_pred_min": round(fn_vals.min() if fn_vals.size > 0 else 0.0, 4),
        "fn_pred_max": round(fn_vals.max() if fn_vals.size > 0 else 0.0, 4),
    }

def evaluate_and_save(model, dataloader, criterion, device, save_csv_path, threshold=0.5):
    model.eval()
    results = []
    with torch.no_grad():
        for images, masks, fnames in tqdm(dataloader, desc="Evaluating"):
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            probs = torch.sigmoid(outputs).cpu().numpy()
            masks_np = masks.cpu().numpy()

            for i in range(images.size(0)):
                pred = probs[i, 0]
                gt = masks_np[i, 0]
                stats = analyze_prediction(pred, gt, threshold=threshold)
                stats.update({"filename": fnames[i], "threshold": threshold})
                results.append(stats)

    df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(save_csv_path), exist_ok=True)
    df.to_csv(save_csv_path, index=False)
    print(f"✅ Saved evaluation results to {save_csv_path}")
    return df

def train_one_epoch(model, dataloader, criterion, optimizer, device, writer, epoch, dataset_name):
    model.train()
    running_loss = 0.0

    all_y_true = []
    all_y_pred = []

    original_images = []
    original_masks = []
    augmented_images = []
    augmented_masks = []
    filenames = []

    # Отображение имя файла → путь
    path_map = {os.path.basename(p): p for p in dataloader.dataset.image_paths}
    mask_map = {os.path.basename(p): p for p in dataloader.dataset.mask_paths}

    for batch_idx, (images, masks, fnames) in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        images = images.to(device)
        masks = masks.to(device)

        if epoch == 0 and batch_idx == 0:
            for i in range(min(5, images.size(0))):
                fname = fnames[i]
                img_path = path_map[fname]
                mask_path = mask_map[fname]

                # --- Загружаем оригинал без аугментации и нормализации ---
                image_np = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
                mask_np = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                image_np = cv2.resize(image_np, (256, 256)) / 255.0
                mask_np = cv2.resize(mask_np, (256, 256))
                mask_np = (mask_np > 127).astype(np.float32)

                image_tensor = torch.from_numpy(image_np.transpose(2, 0, 1)).float()
                mask_tensor = torch.from_numpy(mask_np).unsqueeze(0)

                original_images.append(image_tensor)
                original_masks.append(mask_tensor)

                # --- Сохраняем то, что реально пошло в модель ---
                augmented_images.append(images[i].detach().cpu())
                augmented_masks.append(masks[i].detach().cpu())
                filenames.append(fname)

            visualize_samples_with_names(
                original_images,
                original_masks,
                augmented_images,  # raw input to model
                augmented_masks,
                filenames,
                epoch=epoch,
                prefix="train"
            )

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = (outputs > 0.5).float().detach()
        all_y_true.append(masks.cpu().numpy())
        all_y_pred.append(preds.cpu().numpy())

    epoch_loss = running_loss / len(dataloader.dataset)
    all_y_true = np.concatenate(all_y_true, axis=0)
    all_y_pred = np.concatenate(all_y_pred, axis=0)

    iou, recall, precision, f1 = metrics(all_y_true, all_y_pred)

    writer.add_scalar(f"Loss/Train/{dataset_name}", epoch_loss, epoch)
    writer.add_scalar(f"Metrics/Train_IoU/{dataset_name}", iou, epoch)
    writer.add_scalar(f"Metrics/Train_Recall/{dataset_name}", recall, epoch)
    writer.add_scalar(f"Metrics/Train_Precision/{dataset_name}", precision, epoch)
    writer.add_scalar(f"Metrics/Train_F1/{dataset_name}", f1, epoch)
    torch.cuda.empty_cache()

    return epoch_loss, iou, recall, precision, f1


dataset = {"name": "40_60", 
     "train_image_dir": 'C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/train/40_60',
     "train_mask_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Masks/train/40_60",
     "val_image_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/val",
     "val_mask_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Masks/val",
     "test_image_dir": "C:/Users/Dara/Desktop/Final_thesis/data_choosing/Images/test",
     "test_mask_dir": "C:/Users/Dara/Desktop/Final_thesis//data_choosing/Masks/test"}

results = []
encoder_name = "resnet34"  

dataset_name = dataset["name"]
print(f"Training on dataset: {dataset_name}")

train_dataset = PolypDataset(dataset["train_image_dir"], dataset["train_mask_dir"], transform=get_train_augmentations())
val_dataset = PolypDataset(dataset["val_image_dir"], dataset["val_mask_dir"])
test_dataset = PolypDataset(dataset["test_image_dir"], dataset["test_mask_dir"])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

model = smp.UnetPlusPlus(
    encoder_name=encoder_name,
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation="sigmoid"
)
model = model.to(device)

criterion = DiceBCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

log_dir = f"logs/{dataset_name}_{encoder_name}"
writer = SummaryWriter(log_dir=log_dir)

patience = 10
best_val_iou = 0.0
best_epoch = -1
epochs_without_improvement = 0

# for epoch in range(num_epochs):
#     print(f"Эпоха {epoch+1}/{num_epochs} для датасета {dataset_name}")
    
#     train_loss, train_iou, train_recall, train_precision, train_f1 = train_one_epoch(
#         model, train_loader, criterion, optimizer, device, writer, epoch, dataset_name)
#     print(f"Train Loss: {train_loss:.4f} | IoU: {train_iou:.4f} | Recall: {train_recall:.4f} | Precision: {train_precision:.4f} | F1: {train_f1:.4f}")
    
#     val_loss, val_iou, val_recall, val_precision, val_f1 = validate(
#         model, val_loader, criterion, device, writer, epoch, dataset_name)
#     print(f"Val Loss: {val_loss:.4f} | IoU: {val_iou:.4f} | Recall: {val_recall:.4f} | Precision: {val_precision:.4f} | F1: {val_f1:.4f}")

#     if val_iou > best_val_iou:
#         best_val_iou = val_iou
#         best_epoch = epoch
#         torch.save(model.state_dict(), f"unetplusplus_{dataset_name}_best_model.pth")
#         print("Новая лучшая модель сохранена.")
#         epochs_without_improvement = 0
#     else:
#         epochs_without_improvement += 1
#         if epochs_without_improvement >= patience:
#             print(f"Ранняя остановка на эпохе {epoch+1}")
#             break


model.load_state_dict(torch.load(f"unetplusplus_{dataset_name}_best_model.pth"))
test_loss, test_iou, test_recall, test_precision, test_f1 = test_model(
    model, test_loader, criterion, device, save_folder=f"predicted_masks_{dataset_name}")

results.append({
    "Dataset": dataset_name,
    "Best Epoch": best_epoch + 1,
    "Test IoU": test_iou,
    "Test Recall": test_recall,
    "Test Precision": test_precision,
    "Test F1": test_f1,
    "Test Loss": test_loss
})

writer.close()

results_df = pd.DataFrame(results)
print(results_df)

Training on dataset: 40_60


C:\Temp\ipykernel_12224\3181544907.py:149: UserWarning: Argument 'fill_value' is not valid and will be ignored.
  A.CoarseDropout(
Evaluating: 100%|██████████| 193/193 [00:26<00:00,  7.34it/s]


✅ Saved evaluation results to results/test_predictions.csv
  Dataset  Best Epoch  Test IoU  Test Recall  Test Precision   Test F1  \
0   40_60           0  0.774491     0.857483        0.888915  0.872916   

   Test Loss  
0   0.527112  


In [27]:
def test_model_with_thresholds(model, dataloader, criterion, device, thresholds=None, base_save_dir="threshold_sweep", dataset_name="test"):

    if thresholds is None:
        thresholds = [round(x * 0.1, 2) for x in range(1, 10)]  # [0.1, 0.2, ..., 0.9]

    summary_metrics = []
    summary_path = os.path.join("results", f"{dataset_name}_threshold_metrics.csv")

    for t in thresholds:
        folder = os.path.join(base_save_dir, f"t{int(t * 100)}")
        os.makedirs(folder, exist_ok=True)

        # TensorBoard логирование
        writer = SummaryWriter(log_dir=os.path.join("logs", f"{dataset_name}_t{int(t*100)}"))

        # Сохраняем предсказания масок как .png + метрики в CSV
        test_loss, test_iou, test_recall, test_precision, test_f1 = test_model(
            model, dataloader, criterion, device,
            save_folder=folder
        )

        # Логируем в TensorBoard
        writer.add_scalar(f"Loss/Test/{dataset_name}", test_loss, t)
        writer.add_scalar(f"Metrics/Test_IoU/{dataset_name}", test_iou, t)
        writer.add_scalar(f"Metrics/Test_Recall/{dataset_name}", test_recall, t)
        writer.add_scalar(f"Metrics/Test_Precision/{dataset_name}", test_precision, t)
        writer.add_scalar(f"Metrics/Test_F1/{dataset_name}", test_f1, t)
        writer.close()

        csv_path = os.path.join("results", f"{dataset_name}_predictions_t{int(t*100)}.csv")
        evaluate_and_save(model, dataloader, criterion, device, save_csv_path=csv_path, threshold=t)

        summary_metrics.append({
            "threshold": t,
            "test_loss": round(test_loss, 5),
            "mean_iou": round(test_iou, 5),
            "mean_recall": round(test_recall, 5),
            "mean_precision": round(test_precision, 5),
            "mean_f1": round(test_f1, 5)
        })

        # Обновляем .csv после каждой итерации
        intermediate_df = pd.DataFrame(summary_metrics)
        intermediate_df.to_csv(summary_path, index=False)

        print(f" Threshold {t:.2f} — Done. Results saved in: {folder} and {csv_path}")

    print(f"\n📊 Summary saved to: {summary_path}")


In [29]:

model.load_state_dict(torch.load(f"unetplusplus_{dataset['name']}_best_model.pth"))
model.to(device)

test_model_with_thresholds(
    model=model,
    dataloader=test_loader,
    criterion=criterion,
    device=device,
    thresholds=[round(x * 0.1, 2) for x in range(1, 10)],  # 0.1–0.9
    base_save_dir="threshold_sweep",
    dataset_name=dataset["name"]
)


Evaluating: 100%|██████████| 193/193 [00:25<00:00,  7.71it/s]


✅ Saved evaluation results to results/test_predictions.csv


Evaluating: 100%|██████████| 193/193 [00:25<00:00,  7.71it/s]


✅ Saved evaluation results to results\40_60_predictions_t10.csv
✅ Threshold 0.10 — Done. Results saved in: threshold_sweep\t10 and results\40_60_predictions_t10.csv


Evaluating: 100%|██████████| 193/193 [00:24<00:00,  7.73it/s]


...

✅ Saved evaluation results to results\40_60_predictions_t80.csv
✅ Threshold 0.80 — Done. Results saved in: threshold_sweep\t80 and results\40_60_predictions_t80.csv


Evaluating: 100%|██████████| 193/193 [00:25<00:00,  7.71it/s]


✅ Saved evaluation results to results/test_predictions.csv


Evaluating: 100%|██████████| 193/193 [00:24<00:00,  7.76it/s]

✅ Saved evaluation results to results\40_60_predictions_t90.csv
✅ Threshold 0.90 — Done. Results saved in: threshold_sweep\t90 and results\40_60_predictions_t90.csv

📊 Summary saved to: results\40_60_threshold_metrics.csv
